In [55]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, RobertaForSequenceClassification, RobertaTokenizer
from transformers import T5ForConditionalGeneration, T5Tokenizer
import pandas as pd
from torch.nn.functional import softmax
from sentence_transformers import SentenceTransformer
import numpy as np


In [18]:
predicted = pd.read_csv("../data/interim/predictions.csv", index_col=0)
predicted = predicted["0"]

predicted.head()

0    Seeing people have so much junk chasing for th...
1                        Once he's dead, they're easy.
2                                       "Pucking hell.
3                     some fools to not pay his bills.
4                         What are you doing on stage?
Name: 0, dtype: object

In [10]:
test = pd.read_csv("../data/interim/test.csv", index_col=0)
reference = test["reference"].head(len(predicted))
translation = test["translation"].head(len(predicted))

test.head()

,reference,translation
568822,Hearing people have so much junk vying for the...,hearing people have a lot of unsolicited atten...
322428,"Once he's dead, they'il be easy.","when he's gone, we'll take them easy."
17886,"""Fucking hell.","""oh, my God."
446836,some fool not to pay his bill.,A looser who can't pay his bill.
56045,"what are you doing on stage, fool?","What are you doing on the stage, weirdo?"


### Toxicity check


In [9]:
toxicity_classifier = RobertaForSequenceClassification.from_pretrained('SkolkovoInstitute/roberta_toxicity_classifier')
toxicity_tokenizer =  RobertaTokenizer.from_pretrained('SkolkovoInstitute/roberta_toxicity_classifier')


Some weights of the model checkpoint at SkolkovoInstitute/roberta_toxicity_classifier were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
def calculate_toxicity(text):
    encoded = toxicity_tokenizer.encode(text, return_tensors='pt')

    logits = toxicity_classifier(encoded)[0][0]
    probs = softmax(logits)

    return probs[1].item()


calculate_toxicity("oh, my God.")

/tmp/ipykernel_31748/361774536.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = softmax(logits)


0.0001319961593253538

In [19]:
predicted_toxicity = predicted.apply(calculate_toxicity)
translation_toxicity = translation.apply(calculate_toxicity)
reference_toxicity = reference.apply(calculate_toxicity)

predicted_toxicity

/tmp/ipykernel_31748/361774536.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = softmax(logits)


0       0.211716
1       0.954745
2       0.971003
3       0.998018
4       0.000042
          ...   
2995    0.993415
2996    0.012222
2997    0.986335
2998    0.000042
2999    0.000064
Name: 0, Length: 3000, dtype: float64

In [20]:
predicted_toxicity.mean()

0.29959127625582915

In [21]:
translation_toxicity.mean()

0.10090807241426349

In [22]:
reference_toxicity.mean()

0.8226643118323991

The mean toxicity score dropped from 0.82 to 0.3, though it is not reached the mean of translation toxicity (labels).

### Similarity check

In [28]:
similarity_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [30]:
def embed(text):
    return similarity_model.encode(text)

In [31]:
predicted_embeddings = predicted.apply(embed)
translation_embeddings = translation.apply(embed)
reference_embeddings = reference.apply(embed)

In [50]:
# Mean predicted and translation similarity
np.array(list(predicted_embeddings.values * translation_embeddings.values)).sum(axis=1).mean()

0.6416714

In [51]:
# Mean reference and translation similarity
np.array(list(reference_embeddings.values * translation_embeddings.values)).sum(axis=1).mean()

0.6998071

The mean similarity of model outputs does not dropped significantly

### Comparison with baseline 
(from here: https://github.com/ramsrigouthamg/Paraphrase-any-question-with-T5-Text-To-Text-Transfer-Transformer-)

In [56]:
baseline_tokenizer = T5Tokenizer.from_pretrained("ramsrigouthamg/t5_paraphraser")

baseline_model = T5ForConditionalGeneration.from_pretrained("ramsrigouthamg/t5_paraphraser")

In [57]:
def predict_baseline(text):
    generated = baseline_model.generate(
            baseline_tokenizer.encode("paraphrase: " + text, return_tensors="pt"),
            num_beams=10, num_return_sequences=1, max_length=64
        )
    
    return baseline_tokenizer.decode(generated[0], skip_special_tokens=True)

In [58]:
baseline_predictions = reference.apply(predict_baseline)

baseline_predictions

568822    Why do people have so much junk vying for thei...
322428                     Once he's dead, they'll be easy.
17886                                        "Fucking Hell.
446836                       Some fool not to pay his bill.
56045                          What are you doing on stage?
                                ...                        
154862         Or maybe 'cause you're still fucking Harold.
370990    I hate this place. I hate this place. I hate t...
360849                   I'll bite her into Bobby's belt. "
432350                              Get your ass back here!
184367    I knew you were full of shit when you said you...
Name: reference, Length: 3000, dtype: object

In [60]:
baseline_toxicity = baseline_predictions.apply(calculate_toxicity)

baseline_toxicity.mean()

/tmp/ipykernel_31748/361774536.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = softmax(logits)


0.7648516015903781

In [61]:
baseline_embeded = baseline_predictions.apply(embed)

# Mean baseline prediction and translation similarity
np.array(list(baseline_embeded.values * translation_embeddings.values)).sum(axis=1).mean()

0.6797197

The proposed model removes toxicity better than the baseline, though baseline preserves more similarity